## Getting data from Strava

https://pythonhosted.org/stravalib/api.html#stravalib.client.Client.get_activity_streams

In [ ]:
import settings
from stravalib.client import Client

client = Client(access_token = settings.STRAVA_ACCESS_TOKEN)

In [2]:
import polyline
import functools
import pickle

#  <Activity id=1732871895 name='22. Slapský triatlon - běh' resource_state=2>,
#  <Activity id=1732801109 name='22. Slapský triatlon - kolo' resource_state=2>,

In [3]:
types = ['time', 'distance', 'latlng', 'altitude', 'velocity_smooth', 'moving', 'grade_smooth']
streams = client.get_activity_streams(1732801109, types=types, series_type='time')

In [ ]:
streams['distance'].data

### Folium

http://python-visualization.github.io/folium/quickstart.html

https://github.com/python-visualization/folium/blob/master/folium/plugins/heat_map.py

https://matplotlib.org/tutorials/colors/colormaps.html

In [4]:
locations = streams['latlng'].data
grades = streams['velocity_smooth'].data

data = zip(locations, grades)

groups = []

last_grade = None
for location, grade in data:
    if not last_grade or abs(abs(last_grade) - abs(grade)) > 0.4:
        groups.append({'grade': grade, 'grades': [], 'locations': []})
    groups[-1:][0]['locations'].append(location)
    groups[-1:][0]['grades'].append(grade)
    last_grade = grade
    
    
import statistics

for group in groups:
    group['grade'] = statistics.median_high(group['grades'])

In [5]:
import folium
from folium import plugins

m = folium.Map(location=[49.7947117,14.3916288], tiles='Stamen Toner', zoom_start=12.2)

def convert_to_hex(rgba_color):
    red = str(hex(int(rgba_color[0]*255)))[2:].capitalize()
    green = str(hex(int(rgba_color[1]*255)))[2:].capitalize()
    blue = str(hex(int(rgba_color[2]*255)))[2:].capitalize()

    if blue=='0':
        blue = '00'
    if red=='0':
        red = '00'
    if green=='0':
        green='00'

    return '#'+ red + green + blue
    
import matplotlib.cm as cm
from matplotlib.colors import Normalize

cmap = cm.bwr
norm = Normalize(vmin=min(grades), vmax=max(grades))

def colorize(grade):
    return convert_to_hex(cmap(norm(grade)))

for group in groups:
    wind_line = folium.PolyLine(
        group['locations'],
        tooltip=str(group['grade']),
        weight=10,
        color=colorize(group['grade'])
    ).add_to(m)

m

In [ ]:
m.save('index.html')